In [4]:
import pandas as pd
import spacy
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.feature_extraction.text import CountVectorizer
import joblib

# Cargar el dataset
df = pd.read_csv('youtoxic_english_1000.csv')

# Preparar los datos
X = df['Text']
y = df['IsToxic']

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Cargar el modelo de SpaCy en inglés
nlp = spacy.load('en_core_web_sm')

# Función para procesar el texto con SpaCy
def process_text(text):
    doc = nlp(text)
    return ' '.join([token.lemma_.lower() for token in doc if not token.is_stop and not token.is_punct])

# Aplicar el procesamiento de texto
X_train_processed = X_train.apply(process_text)
X_test_processed = X_test.apply(process_text)

# Vectorizar el texto
vectorizer = CountVectorizer()
X_train_vectorized = vectorizer.fit_transform(X_train_processed)
X_test_vectorized = vectorizer.transform(X_test_processed)

# Entrenar el modelo Random Forest
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train_vectorized, y_train)

# Calcular el rendimiento en el conjunto de entrenamiento
y_train_pred = rf_model.predict(X_train_vectorized)
train_accuracy = accuracy_score(y_train, y_train_pred)

# Calcular el rendimiento en el conjunto de prueba
y_test_pred = rf_model.predict(X_test_vectorized)
test_accuracy = accuracy_score(y_test, y_test_pred)

print("Precisión en el conjunto de entrenamiento:", train_accuracy)
print("Precisión en el conjunto de prueba:", test_accuracy)

print("\nInforme de clasificación en el conjunto de prueba:")
print(classification_report(y_test, y_test_pred))

# Calcular la diferencia de rendimiento
overfitting = train_accuracy - test_accuracy
print(f"\nDiferencia de precisión (overfitting): {overfitting:.4f}")

Precisión en el conjunto de entrenamiento: 1.0
Precisión en el conjunto de prueba: 0.725

Informe de clasificación en el conjunto de prueba:
              precision    recall  f1-score   support

       False       0.65      0.87      0.75        93
        True       0.84      0.60      0.70       107

    accuracy                           0.72       200
   macro avg       0.75      0.73      0.72       200
weighted avg       0.75      0.72      0.72       200


Diferencia de precisión (overfitting): 0.2750


In [5]:
# Guardar el modelo y el vectorizador
joblib.dump(rf_model, 'esencial_model.joblib')
joblib.dump(vectorizer, 'esencial_vectorizer.joblib')


['esencial_vectorizer.joblib']